In [9]:
import yfinance as yf
import pandas as pd
import os

# 株価データ

In [3]:
# Apple関連の銘柄リスト
tickers = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "GOOGL", # Alphabet (Google)
    "AMZN",  # Amazon
    "META",  # Meta Platforms (Facebook)
    "SONY",  # Sony
    "6758.T", # Sony (Tokyo Stock Exchange)
    # "TSMC",  # Taiwan Semiconductor Manufacturing Company
    "2330.TW", # TSMC (Taiwan)
    "QCOM",  # Qualcomm
    "AVGO",  # Broadcom
    "005930.KS", # Samsung Electronics (Korea)
    "GLW",   # Corning
    "2317.TW", # Foxconn (Hon Hai Precision)
    "002475.SZ", # Luxshare Precision
    "BABA",  # Alibaba (China)
    "1810.HK", # Xiaomi (Hong Kong)
    "BBY",   # Best Buy
    "WMT",   # Walmart
    "SPY",   # S&P 500 ETF (for market-wide trends)
    "QQQ",   # Nasdaq 100 ETF
    "VGT",   # Vanguard Information Technology ETF
]


In [4]:
# 保存先のディレクトリ
save_dir = "Stock_DATA"  # 保存するディレクトリ名
os.makedirs(save_dir, exist_ok=True)  # ディレクトリがなければ作成

# 各銘柄のデータを取得して保存
for ticker in tickers:

    data = yf.download(ticker, start="1999-12-29", end="2024-01-03")  # 期間を指定

    # データが空かどうかをチェック
    if data.empty:
        print(f"No data found for {ticker}, skipping...")
        continue  # データが空なら次の銘柄に進む

    # 日付をYYYY-MM-DD形式に変換
    data.reset_index(inplace=True)  # インデックス（日付）を列に変換
    data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')  # 日付をYYYY-MM-DD形式に変換

    # 必要な列のみを残す
    data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

    data.columns = [col[0] for col in data.columns]

    # CSVファイルとして保存（インデックスとして日付を含めて保存）
    file_path = os.path.join(save_dir, f"{ticker}.csv")
    data.to_csv(file_path, index=False)  # index=Falseでインデックスは保存しない

    print(f"Completed: {ticker}")

[*********************100%***********************]  1 of 1 completed


Completed: AAPL


[*********************100%***********************]  1 of 1 completed


Completed: MSFT


[*********************100%***********************]  1 of 1 completed


...

Completed: SPY


[*********************100%***********************]  1 of 1 completed


Completed: QQQ


[*********************100%***********************]  1 of 1 completed

Completed: VGT


In [11]:
save_dir = "Formatted_DATA"  # 保存するディレクトリ名

folder_path = "Stock_DATA"  # フォルダのパス
file_list = os.listdir(folder_path)  # フォルダ内のファイルリスト

for file in file_list:
    # CSVファイルの読み込み
    file_path = f"{folder_path}/{file}"  # 保存されたCSVファイルのパス
    data = pd.read_csv(file_path)

    # Dateカラムを日付型に変換
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

    # 最小の日付を取得
    min_date = data['Date'].min()

    # 2000-01-01から2024-01-01の範囲の日付を作成
    start_date = pd.to_datetime("2000-01-01")
    end_date = pd.to_datetime("2024-01-01")
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

    # DataFrameのインデックスをDateに設定
    data.set_index('Date', inplace=True)

    # 欠損している日付を補完（再インデックスして補完）
    data = data.reindex(full_date_range)

    # 欠損値の補完（前方向補完、後方向補完、線形補完など）
    data.ffill(inplace=True)  # 前方向補完
    data.bfill(inplace=True)  # 後方向補完
    data.interpolate(method='linear', inplace=True)  # 線形補完

    # インデックスを日付から列に戻す
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'Date'}, inplace=True)

    # 最小の日付より前の行のデータを0にする
    data.loc[data['Date'] < min_date, ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = 0

    # 修正後のデータを新しいCSVファイルに保存
    output_file_path = f"{save_dir}/{file}"
    data.to_csv(output_file_path, index=False)

/var/folders/nz/pdh3wbl139nfbnd6j9xbwx300000gn/T/ipykernel_81409/883697039.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.ffill(inplace=True)  # 前方向補完


In [ ]:
# 指数データを取得（例: S&P500）
sp500_data = yf.download("^GSPC", start="2000-01-01", end="2024-01-01")

print(sp500_data)

In [ ]:
# 取得する通貨ペア（EUR/USD）
currency_pair = 'EURUSD=X'

# 1年間の為替データを取得
data = yf.download(currency_pair, start="2000-01-01", end="2024-01-01")

# 取得したデータを表示
print(data)


In [ ]:
# 米国10年国債利回りのティッカーシンボル
bond_ticker = '^TNX'

# 1年間の国債利回りデータを取得
data = yf.download(bond_ticker, start="2000-01-01", end="2024-01-01")

# データを表示
print(data)
